In [1]:
import pandas as pd
import pickle

In [2]:
from tableone import TableOne

# Convert 'scheduled_for_dttm' column in r_df to datetime
r_df['scheduled_for_dttm'] = pd.to_datetime(r_df['scheduled_for_dttm'], utc=True)

# Define the date ranges for training, validation, and test sets
train_start_date = pd.to_datetime('2016-01-02').tz_localize('UTC')
train_end_date = pd.to_datetime('2021-10-05').tz_localize('UTC')
val_start_date = pd.to_datetime('2021-10-05').tz_localize('UTC')
val_end_date = pd.to_datetime('2022-05-17').tz_localize('UTC')
test_start_date = pd.to_datetime('2022-05-17').tz_localize('UTC')
test_end_date = pd.to_datetime('2022-12-31').tz_localize('UTC')

# Create masks for each dataset based on the date ranges in r_df
train_mask_r = (r_df['scheduled_for_dttm'] >= train_start_date) & (r_df['scheduled_for_dttm'] <= train_end_date)
val_mask_r = (r_df['scheduled_for_dttm'] > val_start_date) & (r_df['scheduled_for_dttm'] <= val_end_date)
test_mask_r = (r_df['scheduled_for_dttm'] > test_start_date) & (r_df['scheduled_for_dttm'] <= test_end_date)



# Define missing columns including the new variables
missing_columns = [
    'enc_id', 'periop_platelets_units_transfused', 'periop_prbc_units_transfused', 
    'periop_ffp_units_transfused', 'periop_cryoprecipitate_units_transfused', 'sched_addon_yn', 
    'sched_or_equip_cell_saver_yn', 'sched_neuromonitoring_yn', 'sched_bypass_yn', 'sex_identity_female', 
    'sex_identity_male', 'sex_identity_transsexfemale', 'sex_identity_transsexmale', 
    'sex_identity_nonbinary', 'sex_identity_other', 'sex_identity_unknown', 'language_interpreter_needed_yn', 
    'race_ethnicity_asian', 'race_ethnicity_black', 'race_ethnicity_latinx', 'race_ethnicity_multi', 
    'race_ethnicity_native_am_alaska', 'race_ethnicity_hi_pac_islander', 'race_ethnicity_other', 'race_ethnicity_swana', 
    'race_ethnicity_unknown', 'home_meds_anticoag_warfarin_yn', 'home_meds_anticoag_heparin_sq_yn', 
    'home_meds_anticoag_heparin_iv_yn', 'arrival_ed_yn', 'hist_transf_1week_yn', 'hist_transf_1day_yn', 
    'hist_prior_transf_yn', 'sched_surgical_service', 'preop_hematocrit', 'preop_platelets', 'preop_inr', 
    'preop_ptt', 'preop_creatinine', 'preop_sodium', 'preop_albumin', 'preop_bilirubin', 'home_meds_anticoag_yn'
]

# Ensure missing columns are present in the DataFrame
available_columns = [col for col in missing_columns if col in r_df.columns]


print("Available columns in the DataFrame:")
print(available_columns)

In [3]:

r_df = pd.read_csv("/test jupyter/data_elective_only.csv")
with open("/test jupyter/train_test_elective_only.pkl", 'rb') as f:
    X_train, y_train, X_valid, y_valid, X_test, y_test = pickle.load(f)

In [7]:
len(X_train.columns)

121

In [8]:
'preop_inr' in X_train.columns

False

In [10]:
X_train.dataset_id.unique()

array([3], dtype=int64)

In [4]:
# Combine features and labels into DataFrames for the pickle data

# Group surgical services into categories
X_train['sched_surgical_service'] = X_train['sched_surgical_service'].apply(lambda x: x if x in ['Cardiac Surgery', 'Vascular Surgery', 'Transplant'] else 'All Other Services')
train_df_p = X_train.copy()
train_df_p['transfusion'] = (y_train['periop_prbc_units_transfused'] > 0).astype(int)  

X_valid['sched_surgical_service'] = X_valid['sched_surgical_service'].apply(lambda x: x if x in ['Cardiac Surgery', 'Vascular Surgery', 'Transplant'] else 'All Other Services')
val_df_p = X_valid.copy()
val_df_p['transfusion'] = (y_valid['periop_prbc_units_transfused'] > 0).astype(int) 

X_test['sched_surgical_service'] = X_test['sched_surgical_service'].apply(lambda x: x if x in ['Cardiac Surgery', 'Vascular Surgery', 'Transplant'] else 'All Other Services')
test_df_p = X_test.copy()
test_df_p['transfusion'] = (y_test['periop_prbc_units_transfused'] > 0).astype(int)  

In [5]:
train_df = pd.concat([train_df_p,y_train],axis=1)
train_df = train_df[train_df.periop_prbc_units_transfused < 700]
val_df = pd.concat([val_df_p,y_valid],axis=1)
test_df = pd.concat([test_df_p,y_test],axis=1)
test_df = test_df[test_df.periop_prbc_units_transfused < 700]

In [6]:
y_train.columns

Index(['periop_platelets_units_transfused', 'periop_prbc_units_transfused',
       'periop_ffp_units_transfused',
       'periop_cryoprecipitate_units_transfused'],
      dtype='object')

In [11]:
print(sum(train_df[train_df['periop_prbc_units_transfused']>0].sex_female))

3371


In [12]:
print(sum(val_df[val_df['periop_prbc_units_transfused']>0].sex_female))

352


In [9]:
print(len(test_df))

23506


In [24]:
 
# Handle missing values more carefully
# For categorical columns, fill NaN with 0
categorical_cols = ['sex_female', 'sex_male','sex_nonbinary', 'sex_unknown', 'race_ethnicity_asian', 'race_ethnicity_black', 'race_ethnicity_latinx',
                    'race_ethnicity_multi', 'race_ethnicity_native_am_alaska', 'race_ethnicity_hi_pac_islander',
                    'race_ethnicity_other', 'race_ethnicity_swana', 'race_ethnicity_unknown',
                    'sched_addon_yn', 'sched_or_equip_cell_saver_yn', 'sched_neuromonitoring_yn', 'sched_bypass_yn',
                    'arrival_ed_yn', 'home_meds_anticoag_yn']

for col in categorical_cols:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna(0)
        val_df[col] = val_df[col].fillna(0)
        test_df[col] = test_df[col].fillna(0)

# For numerical columns, we'll fill NaN with median
numerical_cols = ['age', 'weight_kg', 'height_cm', 'bmi', 'sched_est_case_length', 'preop_hematocrit', 'preop_platelets',
                  'preop_inr', 'preop_ptt', 'preop_creatinine', 'preop_sodium', 'preop_albumin', 'preop_bilirubin']

for col in numerical_cols:
    if col in train_df.columns:
        median_value = train_df[col].median()
        train_df[col] = train_df[col].fillna(median_value)
        val_df[col] = val_df[col].fillna(median_value)
        test_df[col] = test_df[col].fillna(median_value)

# Improved function to combine sex columns into one
def combine_sex_columns(row):
    sex_columns = ['sex_female', 'sex_male',   'sex_nonbinary', 'sex_unknown']
    
    sexs = [col.split('_')[-1] for col in sex_columns if col in row.index and row[col] == 1]
    
    if not sexs:
        return 'unknown'
    elif len(sexs) == 1:
        return sexs[0]
    else:
        return 'multi'

# Improved function to combine race columns into one
def combine_race_columns(row):
    race_columns = ['race_ethnicity_asian', 'race_ethnicity_black', 'race_ethnicity_latinx',
                    'race_ethnicity_multi', 'race_ethnicity_native_am_alaska', 'race_ethnicity_hi_pac_islander',
                    'race_ethnicity_other', 'race_ethnicity_swana', 'race_ethnicity_unknown']
    
    races = [col.split('_')[-1] for col in race_columns if col in row.index and row[col] == 1]
    
    if not races:
        return 'white'  # Assuming white if no other race is specified
    elif len(races) == 1:
        return races[0]
    else:
        return 'multi'

# Apply the functions to combine sex and race columns
train_df['sex'] = train_df.apply(combine_sex_columns, axis=1)
val_df['sex'] = val_df.apply(combine_sex_columns, axis=1)
test_df['sex'] = test_df.apply(combine_sex_columns, axis=1)

train_df['race'] = train_df.apply(combine_race_columns, axis=1)
val_df['race'] = val_df.apply(combine_race_columns, axis=1)
test_df['race'] = test_df.apply(combine_race_columns, axis=1)

# Define the columns for TableOne
columns = [
    'age', 'sex', 'race', 'prepare_asa', 'prior_dept_inpt_yn', 'icu_admit_prior_24hr_yn', 'arrival_ed_yn', 
    'weight_kg', 'height_cm', 'bmi', 'periop_platelets_units_transfused', 'periop_prbc_units_transfused', 
    'periop_ffp_units_transfused', 'periop_cryoprecipitate_units_transfused', 'sched_est_case_length', 
    'sched_surgeon_cnt', 'sched_proc_cnt', 'sched_proc_diag_cnt', 'sched_addon_yn', 'sched_or_equip_cell_saver_yn', 
    'sched_neuromonitoring_yn', 'sched_bypass_yn', 'sched_surgical_service', 'preop_hematocrit', 'preop_platelets', 
    'preop_inr', 'preop_ptt', 'preop_creatinine', 'preop_sodium', 'preop_albumin', 'preop_bilirubin', 
    'home_meds_anticoag_yn'
]

# Define categorical columns
categorical = [
    'sex', 'race', 'prior_dept_inpt_yn', 'icu_admit_prior_24hr_yn', 'arrival_ed_yn', 'sched_addon_yn', 
    'sched_or_equip_cell_saver_yn', 'sched_neuromonitoring_yn', 'sched_bypass_yn', 'sched_surgical_service', 
    'home_meds_anticoag_yn'
]

# Filter columns and categorical columns to include only those available in the DataFrame
available_columns = [col for col in columns if col in train_df.columns]
available_categorical = [col for col in categorical if col in train_df.columns]

# Define the groupby column
groupby = 'transfusion'



In [25]:
# Data validation steps
print("Checking for missing values in race and sex columns:")
race_sex_columns = [col for col in train_df.columns if col.startswith(('race_ethnicity_', 'sex_'))]
print(train_df[race_sex_columns].isnull().sum())

print("\nDistribution of values in race and sex columns:")
print(train_df[race_sex_columns].sum())

print("\nDistribution of values in new 'race' column:")
print(train_df['race'].value_counts(normalize=True))

print("\nDistribution of values in new 'sex' column:")
print(train_df['sex'].value_counts(normalize=True))

# Create TableOne for the training set
table_train = TableOne(train_df, columns=available_columns, categorical=available_categorical, groupby=groupby, pval=True,
                      pval_adjust='bonferroni')#htest_name=True)
#print("\nTraining set TableOne:")
#print(table_train.tabulate(tablefmt="github", showindex=False, headers="keys"))

# Create TableOne for the validation set
table_val = TableOne(val_df, columns=available_columns, categorical=available_categorical, groupby=groupby, pval=True,
                    pval_adjust='bonferroni')
print("\nValidation set TableOne:")
print(table_val.tabulate(tablefmt="github", showindex=False, headers="keys"))

# Create TableOne for the test set
table_test = TableOne(test_df, columns=available_columns, categorical=available_categorical, groupby=groupby, pval=True,
                     pval_adjust='bonferroni')
#print("\nTest set TableOne:")
#print(table_test.tabulate(tablefmt="github", showindex=False, headers="keys"))

Checking for missing values in race and gender columns:
sex_female                         0
sex_male                           0
sex_nonbinary                      0
sex_unknown                        0
race_ethnicity_asian               0
race_ethnicity_black               0
race_ethnicity_latinx              0
race_ethnicity_multi               0
race_ethnicity_native_am_alaska    0
race_ethnicity_hi_pac_islander     0
race_ethnicity_other               0
race_ethnicity_swana               0
race_ethnicity_unknown             0
race_ethnicity_white               0
dtype: int64

Distribution of values in race and gender columns:
sex_female                         101512
sex_male                            86530
sex_nonbinary                           0
sex_unknown                             0
race_ethnicity_asian                25673
race_ethnicity_black                10729
race_ethnicity_latinx               25795
race_ethnicity_multi                 4768
race_ethnicity_native_am_

In [26]:
table_train.to_excel('train.xlsx')
table_val.to_excel('val.xlsx')
table_test.to_excel('test.xlsx')

In [27]:
overall_table = pd.concat([train_df,val_df,test_df])

In [28]:

overall = TableOne(overall_table, columns=available_columns, categorical=available_categorical, groupby=groupby, pval=True,
                    pval_adjust='bonferroni')

overall.to_excel('all.xlsx')

In [9]:

data_df = pd.read_csv("/Marlenelin/data_elective_only_485ae5e.csv")

 
print('SUMMARY OF DATA')
print('Total # Cases :' + str(len(data_df)) )
print('Missing:')

time_points = [
    'an_hour_before_last_score', 'an_hour_before_max_score',
    'three_pm_day_before_last_score', 'three_pm_day_before_max_score',
    'seven_days_before_last_score', 'seven_days_before_max_score',
    'month_before_last_score', 'month_before_max_score']

print(data_df[time_points].isna().sum())

# Identify rows with missing values in any race/ethnicity column # Create a new DataFrame with only those rows
if 0:
    missing_race_ethnicity_rows = data_df[time_points].apply(lambda row: row.isna().any(), axis=1)
    data_df = data_df[~missing_race_ethnicity_rows] #538 cases with missing r/e b/c they are the NORA add on cases. 
    print("Start Date: " + str(data_df['anes_start'].min()) + " End Date: " + str(data_df['anes_start'].max()))
print('Total # Cases with NORA Removed: ' + str(len(data_df)) )
p_df = data_df.dropna(subset=[ 'an_hour_before_last_score'])
len(p_df)

SUMMARY OF DATA
Total # Cases :24932
Missing:
an_hour_before_last_score          1713
an_hour_before_max_score           1713
three_pm_day_before_last_score     2291
three_pm_day_before_max_score      2291
seven_days_before_last_score       6076
seven_days_before_max_score        6076
month_before_last_score           14570
month_before_max_score            14570
dtype: int64
Total # Cases with NORA Removed: 24932


23219

In [10]:
 
# Assuming p_df and pred are already loaded
# Merge p_df with pred on enc_id
p_df['target'] = p_df['periop_prbc_units_transfused']>0

# Convert 'scheduled_for_dttm' column in p_df to datetime if needed
p_df['scheduled_for_dttm'] = pd.to_datetime(p_df['scheduled_for_dttm'], utc=True)

# Group surgical services into categories
p_df['sched_surgical_service'] = p_df['sched_surgical_service'].apply(lambda x: x if x in ['Cardiac Surgery', 'Vascular Surgery', 'Transplant'] else 'All Other Services')

# Improved function to combine sex columns into one
def combine_sex_columns(row):
    sex_columns = ['sex_female', 'sex_male', 'sex_nonbinary', 'sex_unknown']
    
    sexs = [col.split('_')[-1] for col in sex_columns if col in row.index and row[col] == 1]
    
    if not sexs:
        return 'unknown'
    elif len(sexs) == 1:
        return sexs[0]
    else:
        return 'multi'

# Improved function to combine race columns into one
def combine_race_columns(row):
    race_columns = ['race_ethnicity_asian', 'race_ethnicity_black', 'race_ethnicity_latinx',
                    'race_ethnicity_multi', 'race_ethnicity_native_am_alaska', 'race_ethnicity_hi_pac_islander',
                    'race_ethnicity_other', 'race_ethnicity_swana', 'race_ethnicity_unknown', 'race_ethnicity_white']
    
    races = [col.split('_')[-1] for col in race_columns if col in row.index and row[col] == 1]
    
    if not races:
        return 'unknown'
    elif len(races) == 1:
        return races[0]
    else:
        return 'multi'

# Apply the functions to combine sex and race columns
p_df['sex'] = p_df.apply(combine_sex_columns, axis=1)
p_df['race'] = p_df.apply(combine_race_columns, axis=1)

# Define the columns for TableOne
columns = [
    'age', 'sex', 'race', 'prepare_asa', 'prior_dept_inpt_yn', 'icu_admit_prior_24hr_yn', 'arrival_ed_yn', 
    'weight_kg', 'height_cm', 'bmi', 'periop_platelets_units_transfused', 'periop_prbc_units_transfused', 
    'periop_ffp_units_transfused', 'periop_cryoprecipitate_units_transfused', 'sched_est_case_length', 
    'sched_surgeon_cnt', 'sched_proc_cnt', 'sched_proc_diag_cnt', 'sched_addon_yn', 'sched_or_equip_cell_saver_yn', 
    'sched_neuromonitoring_yn', 'sched_bypass_yn', 'sched_surgical_service', 'preop_hematocrit', 'preop_platelets', 
    'preop_creatinine', 'preop_sodium', 'home_meds_anticoag_warfarin_yn', 'pred_ts_abo', 'pred_rbc', 'pred_ts'
]

# Define categorical columns
categorical = [
    'sex', 'race', 'prior_dept_inpt_yn', 'icu_admit_prior_24hr_yn', 'arrival_ed_yn', 'sched_addon_yn', 
    'sched_or_equip_cell_saver_yn', 'sched_neuromonitoring_yn', 'sched_bypass_yn', 'sched_surgical_service', 
    'home_meds_anticoag_warfarin_yn'
]

# Filter columns and categorical columns to include only those available in the DataFrame
available_columns = [col for col in columns if col in p_df.columns]
available_categorical = [col for col in categorical if col in p_df.columns]

# Define the groupby column (now using 'target')
groupby = 'target'

# Handle missing values
# For categorical columns, fill NaN with 0
for col in available_categorical:
    p_df[col] = p_df[col].fillna(0)

# For numerical columns, fill NaN with median
numerical_cols = [col for col in available_columns if col not in available_categorical]
for col in numerical_cols:
    p_df[col] = p_df[col].fillna(p_df[col].median())

# Create TableOne
table = TableOne(p_df, columns=available_columns, categorical=available_categorical, groupby=groupby, pval=True)

# Print the table
print("\nTableOne:")
print(table.tabulate(tablefmt="github", showindex=False, headers="keys"))

# Print missing values
missing_values = p_df[available_columns].isnull().sum()
print("\nMissing Values:")
print(missing_values)
table.to_excel('prosp.xlsx')

C:\Users\mlin2\AppData\Local\Temp\3\ipykernel_21000\123752628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_df['target'] = p_df['periop_prbc_units_transfused']>0
C:\Users\mlin2\AppData\Local\Temp\3\ipykernel_21000\123752628.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_df['scheduled_for_dttm'] = pd.to_datetime(p_df['scheduled_for_dttm'], utc=True)
C:\Users\mlin2\AppData\Local\Temp\3\ipykernel_21000\123752628.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from


TableOne:
| ('', '')           | ('Grouped by target', 'Missing')   | ('Grouped by target', 'Overall')   | ('Grouped by target', 'False')   | ('Grouped by target', 'True')   | ('Grouped by target', 'P-Value')   |
|--------------------|------------------------------------|------------------------------------|----------------------------------|---------------------------------|------------------------------------|
|                    |                                    | 23219                              | 22695                            | 524                             |                                    |
|                    | 0                                  | 57.3 (17.4)                        | 57.2 (17.4)                      | 61.1 (15.4)                     | <0.001                             |
| female             |                                    | 12087 (52.1)                       | 11809 (52.0)                     | 278 (53.1)                      | 0.676      